Connect to the SQL database.

In [42]:
import duckdb
import pandas as pd
import ipywidgets
import os

os.chdir('/work/users/c/a/cannecar/co_analysis/seq_pipeline/workflow/scripts/')
%reload_ext sql

%config SqlMagic.autopandas = False 
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///vcf_dfs_new.db

In [43]:
%%sql
SET enable_progress_bar=false;
SET preserve_insertion_order = false;
SET memory_limit=`{snakemake.resources['mem_mb'] / 1000 - 2};

Success


Define the locations of the references and samples, and quality/depth cutoffs.

In [44]:
ref_parent_file = "../../data/parquets/w1118.parquet"
alt_parent_file = "../../data/parquets/oregonr.parquet"
sample_file_glob = "../../data/parquets/WT-G0-0*.parquet"
ref_parent_name = "w1118"
alt_parent_name = "oregonr"

ref_qual_cutoff = 150
ref_depth_cutoff = 0

ref_out_file = "../../data/parquets/reference.parquet2"
sample_out_file = "../../data/parquets/progeny.parquet2"

Import the reference files as a table. 

In [45]:
%%sql
CREATE
OR REPLACE VIEW parents AS
SELECT
    sample,
    chromosome,
    CAST(position AS INTEGER) AS int_pos,
    quality,
    genotype,
    depth,
    allele_depth,
    UPPER(reference) AS up_reference,
    UPPER(variant) AS up_variant,
    (CASE WHEN variant='.' THEN reference ELSE UPPER(variant) END) AS mod_variant
FROM
    read_parquet(['{{ref_parent_file}}', '{{alt_parent_file}}']);


Count


Isolate sites where there are 2 variants (meaning one parent is different from the other).

In [46]:
%%sql
CREATE
OR REPLACE VIEW check_unique_variants AS
SELECT
    chromosome,
    int_pos,
    COUNT(up_variant) AS n_variants
FROM
    parents
GROUP BY
    chromosome,
    int_pos
HAVING
    n_variants >= 2;

Count


Create new reference from the reference parent.

In [47]:
%%sql
CREATE
OR REPLACE VIEW temp_ref AS
SELECT
    sample,
    parents.chromosome,
    parents.int_pos,
    up_reference AS reference,
    mod_variant,
    quality,
    genotype,
    depth,
    allele_depth
FROM
    parents
    INNER JOIN check_unique_variants ON (
        parents.chromosome = check_unique_variants.chromosome
        AND parents.int_pos = check_unique_variants.int_pos
    )
WHERE
    genotype != '0/1'
    AND quality > '{{ref_qual_cutoff}}'
    AND LENGTH (mod_variant) <= 1
    AND LENGTH (up_reference) <= 1
    AND depth > '{{ref_depth_cutoff}}';

Count


In [48]:
%%sql
CREATE
OR REPLACE VIEW ref_parent AS
SELECT
    sample,
    chromosome,
    int_pos,
    mod_variant AS ref_allele
FROM
    temp_ref
WHERE
    sample = '{{ref_parent_name}}';

CREATE
OR REPLACE VIEW alt_parent AS
SELECT
    sample,
    chromosome,
    int_pos,
    mod_variant AS alt_allele
FROM
    temp_ref
WHERE
    sample = '{{alt_parent_name}}';

Count


In [49]:
%%sql
CREATE
OR REPLACE TABLE ref AS
SELECT
    *
FROM
    ref_parent
    INNER JOIN alt_parent ON (
        ref_parent.chromosome = alt_parent.chromosome
        AND ref_parent.int_pos = alt_parent.int_pos
    );

Count


In [50]:
%%sql
CREATE
OR REPLACE TABLE refs_unique AS
SELECT
    *
FROM
    ref
WHERE
    ref_allele != alt_allele;

Count


In [51]:
%%sql
SELECT * FROM refs_unique;

sample,chromosome,int_pos,ref_allele,sample_1,chromosome_1,int_pos_1,alt_allele
w1118,chr3L,2139537,T,oregonr,chr3L,2139537,A
w1118,chr3R,27358334,A,oregonr,chr3R,27358334,G
w1118,chr2R,11149998,G,oregonr,chr2R,11149998,A
w1118,chr2R,11908355,A,oregonr,chr2R,11908355,G
w1118,chr3L,6300347,C,oregonr,chr3L,6300347,T
w1118,chr2R,23675664,C,oregonr,chr2R,23675664,T
w1118,chr2R,11116846,A,oregonr,chr2R,11116846,G
w1118,chr3R,9867954,A,oregonr,chr3R,9867954,C
w1118,chr2R,18533465,G,oregonr,chr2R,18533465,T
w1118,chr3L,1784845,A,oregonr,chr3L,1784845,T


In [52]:
%%sql

CREATE
OR REPLACE VIEW temp_vcfs AS
SELECT
    *
FROM
    read_parquet ('{{sample_file_glob}}');

Count


In [53]:
%%sql

CREATE
OR REPLACE VIEW vcfs AS
SELECT
    sample,
    chromosome,
    CAST(position AS INTEGER) AS int_pos,
    reference,
    variant,
    quality,
    genotype,
    depth,
    allele_depth,
    (
        CASE
            WHEN temp_vcfs.variant = '.' THEN UPPER(temp_vcfs.reference)
            ELSE UPPER(temp_vcfs.variant)
        END
    ) AS new_variant
FROM
    temp_vcfs;

Count


In [54]:
%%sql

CREATE
OR REPLACE VIEW samples_rearranged AS
SELECT
    *,
    (
        CASE
            WHEN ref_allele = new_variant THEN '.'
            ELSE new_variant
        END
    ) AS var_adjusted
FROM
    vcfs
    INNER JOIN refs_unique ON vcfs.chromosome = refs_unique.chromosome
    AND vcfs.int_pos = refs_unique.int_pos
WHERE
    new_variant = ref_allele
    OR new_variant = alt_allele;

Count


In [57]:
%%sql
SET reserve_insertion_order = true;
CREATE
OR REPLACE TABLE final_samples AS
SELECT 
    string_split(sample, '-')[1] AS condition,
    string_split(sample, '-')[2] AS sample_type,
    string_split(sample, '-')[3] AS sample_num,
    UPPER(ref_allele) AS reference,
    UPPER(var_adjusted) AS variant,
    chromosome,
    int_pos AS position,
    CAST(string_split(allele_depth, ',')[1] AS INTEGER) AS ref_reads,
    CAST(string_split(allele_depth, ',')[2] AS INTEGER) AS variant_reads,
    quality AS QUAL,
    (CASE WHEN var_adjusted='.' AND genotype='1/1' THEN '0/0' WHEN var_adjusted !='.' AND genotype='0/0' THEN '1/1' ELSE genotype END) AS GT,
    depth AS DP
FROM
    samples_rearranged
ORDER BY
    sample_num, chromosome, position;

CHECKPOINT;

Success


In [58]:
%%sql

COPY (SELECT * FROM final_samples)
TO '{{sample_out_file}}'
(FORMAT 'parquet');

COPY (SELECT chromosome, int_pos AS position, ref_allele AS reference, alt_allele AS variant FROM ref)
TO '{{ref_out_file}}'
(FORMAT 'parquet');


Count


In [64]:
master_df = pd.read_parquet(sample_out_file)
master_df = master_df.fillna(0)
master_df['ref_reads'] = master_df['ref_reads'].astype(int)
master_df['variant_reads'] = master_df['variant_reads'].astype(int)
master_df

,condition,sample_type,sample_num,reference,variant,chromosome,position,ref_reads,variant_reads,QUAL,GT,DP
0,WT,G0,001,G,A,chr2L,7902,68,0,234.996002,1/1,68
1,WT,G0,001,A,G,chr2L,8263,74,0,252.994995,1/1,74
2,WT,G0,001,G,A,chr2L,27181,65,0,225.996002,1/1,65
3,WT,G0,001,C,G,chr2L,31003,58,0,204.996002,1/1,58
4,WT,G0,001,A,C,chr2L,32124,102,0,283.235992,1/1,102
...,...,...,...,...,...,...,...,...,...,...,...,...
1816974,WT,G0,016,C,.,chrY,2871752,4,0,41.356800,0/0,4
1816975,WT,G0,016,G,.,chrY,3246517,17,0,64.896599,0/0,17
1816976,WT,G0,016,G,G,chrY,3246517,17,0,64.896599,1/1,17
1816977,WT,G0,016,A,.,chrY,3519285,7,0,48.873699,0/0,7


In [ ]:
def run_TIGER_pipeline(master_dataframe, project_dir: str):

    #try to make TIGER inputs


    if project_dir not in os.listdir(): # if project folder not made...

        print('making TIGER inputs...')
        create_TIGER_inputs_and_directory(master_dataframe, new_project_dir=project_dir) #make inputs, project directory, and sample sub-directories

        for sample_dir in os.listdir(project_dir):

            sample_folder = os.getcwd()+'/'+project_dir+'/'+sample_dir+'/'

            if sample_dir != '.DS_Store':

                if sample_dir+'.tiger_input.txt' in os.listdir(sample_folder) and sample_dir+'.CO_estimates.txt' not in os.listdir(sample_folder): #if folder has input but missing this TIGER output, do TIGER

                    subprocess.run(['sh', 'run_TIGER.sh', os.getcwd()+'/'+project_dir+'/'+sample_dir+'/', sample_dir])

                elif sample_dir+'.tiger_input.txt' in os.listdir(sample_folder) and sample_dir+'.CO_estimates.txt' in os.listdir(sample_folder): #if folder has input and has this TIGER output, skip...
                    print(sample_dir, 'already processed...')
                    continue

                elif sample_dir+'.tiger_input.txt' not in os.listdir(sample_folder): #warn if input not in folder for some reason
                    print(sample_dir, 'needs TIGER input...')



    elif project_dir in os.listdir(): #if project directory alredy exists
        for sample_dir in os.listdir(project_dir): #for every sample directory, run tiger

            if sample_dir != '.DS_Store': #skip mac's DS_store file...useless

                sample_folder = os.getcwd()+'/'+project_dir+'/'+sample_dir+'/'

                if sample_dir+'.tiger_input.txt' in os.listdir(sample_folder) and sample_dir+'.CO_estimates.txt' not in os.listdir(sample_folder): #if folder has input but missing this TIGER output, do TIGER

                    subprocess.run(['sh', 'run_TIGER.sh', os.getcwd()+'/'+project_dir+'/'+sample_dir+'/', sample_dir])

                elif sample_dir+'.tiger_input.txt' in os.listdir(sample_folder) and sample_dir+'.CO_estimates.txt' in os.listdir(sample_folder): #if folder has input and has this TIGER output, skip...
                    print(sample_dir, 'already processed...')
                    continue

                elif sample_dir+'.tiger_input.txt' not in os.listdir(sample_folder): #warn if input not in folder for some reason
                    print(sample_dir, 'needs TIGER input...')